## The final (preliminary) model training/testing using the final dataset for this milestone can be found under the "New Data" heading

In [15]:
import numpy as np
import pandas as pd
import os
import keras

In [2]:
current_path = os.getcwd()
os.chdir(current_path)
print(f"Working directory set to: {current_path}")

Working directory set to: /sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project


In [3]:
import pandas as pd
import glob

# Get all CSV file paths
csv_files = glob.glob("/sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project/final_data/*.csv")

csv_files_sorted = sorted(csv_files)

# Read and concatenate all CSV files into a single DataFrame
dataframes = [pd.read_csv(file) for file in csv_files_sorted]
combined_df = pd.concat(dataframes, ignore_index=True)

# Inspect the combined DataFrame
#combined_df.head()

In [4]:
X_data = combined_df[['insurance','race','marital_status','gender','anchor_age','value','amount','amountuom','label','sequence_num']]

In [5]:
data_encoded = pd.get_dummies(X_data, columns=['insurance','race','marital_status','gender','amountuom','label','value'])

In [6]:
#NaN values in amount
data_encoded.isna().sum()

anchor_age                      0
amount                      15856
sequence_num                    0
insurance_Medicaid              0
insurance_Medicare              0
                            ...  
value_SIMV/PSV/AutoFlow         0
value_SPONT                     0
value_Sensor Medic (HFO)        0
value_Standby                   0
value_VOL/AC                    0
Length: 104, dtype: int64

In [7]:
 data_encoded['amount'] = data_encoded['amount'].fillna(0)

In [8]:
data_encoded.isna().sum().sum()

0

In [9]:
from sklearn.model_selection import train_test_split

# Split into features (X) and target labels (y)
X = data_encoded.values  
X = X.astype(np.float32)
y = combined_df['Died'].values  
y = y.astype(np.int32)

# Split data into training, validation, and testing sets (e.g., 70% train, 15% val, 15% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Testing set size:", len(X_test))

Training set size: 850479
Validation set size: 182245
Testing set size: 182246


In [10]:
import tensorflow as tf

# Define a function to convert arrays to TensorFlow Datasets
def to_tf_dataset(X, y, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=len(X)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

#def to_tf_dataset(X, y, batch_size=32):
#    dataset = tf.data.Dataset.from_tensor_slices((X, y))
 #   dataset = dataset.shuffle(buffer_size=len(X)).padded_batch(
#        batch_size,
#        padded_shapes=([104,], []),
#        padding_values=(0.0, 0)
#        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#    return dataset

# Create TensorFlow Datasets
train_dataset = to_tf_dataset(X_train, y_train)
val_dataset = to_tf_dataset(X_val, y_val)
test_dataset = to_tf_dataset(X_test, y_test)


2024-11-09 16:40:43.465303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31027 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:af:00.0, compute capability: 7.0


In [11]:
# Inspect the dataset
for x, y in train_dataset.take(1):
    print(x, y)

tf.Tensor(
[[ 70.  74.  99. ...   0.   0.   0.]
 [ 67. 124. 223. ...   0.   0.   0.]
 [ 79.  80.  78. ...   0.   0.   0.]
 ...
 [ 77.  88. 451. ...   0.   0.   0.]
 [ 87.  11. 351. ...   0.   0.   0.]
 [ 67.  80.  55. ...   0.   0.   0.]], shape=(32, 104), dtype=float32) tf.Tensor([0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0], shape=(32,), dtype=int32)


2024-11-08 15:47:41.791040: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
element_spec = train_dataset.element_spec
print(element_spec)
#train_dataset.tf.shape

(TensorSpec(shape=(None, 104), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [13]:
#model 1
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1,input_shape = (None,1))
])

optimizer = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer,
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)]
             )
history = model.fit(train_dataset, epochs=10,
                    validation_data=(val_dataset))

/home/rmd9ev/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


I0000 00:00:1731098886.191180  875957 service.cc:146] XLA service 0x56304da2a560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731098886.191233  875957 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-11-08 15:48:06.220152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-08 15:48:06.341418: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8904


   43/26578 ━━━━━━━━━━━━━━━━━━━━ 1:44 4ms/step - binary_accuracy: 0.7550 - loss: 2.1356

I0000 00:00:1731098886.965833  875957 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26578/26578 ━━━━━━━━━━━━━━━━━━━━ 110s 4ms/step - binary_accuracy: 0.7512 - loss: 1.8763 - val_binary_accuracy: 0.7510 - val_loss: 0.5616
Epoch 2/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7507 - loss: 0.5617 - val_binary_accuracy: 0.7510 - val_loss: 0.5613
Epoch 3/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7506 - loss: 0.5618 - val_binary_accuracy: 0.7510 - val_loss: 0.5615
Epoch 4/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7506 - loss: 0.5617 - val_binary_accuracy: 0.7510 - val_loss: 0.5612
Epoch 5/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7509 - loss: 0.5614 - val_binary_accuracy: 0.7510 - val_loss: 0.5615
Epoch 6/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 109s 4ms/step - binary_accuracy: 0.7510 - loss: 0.5613 - val_binary_accuracy: 0.7510 - val_loss: 0.5612
Epoch 7/10
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - binary_accuracy: 0.7512 - loss: 0.5611 - val_binary_accuracy: 0

In [14]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - binary_accuracy: 0.7530 - loss: 0.5593
Loss:  0.5597178339958191
Accuracy:  0.7525377869606018


In [11]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(None,104)), 
    keras.layers.Masking(mask_value=0.0, input_shape=(None, 1)),  # for variable-length sequences
    keras.layers.SimpleRNN(32),  # SimpleRNN with 32 units
    keras.layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_dataset, epochs=5,
                    validation_data=(val_dataset))        
    


/home/rmd9ev/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/home/rmd9ev/.local/lib/python3.10/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 104), dtype=float32). Expected shape (None, None, 104), but input has incompatible shape (None, 104)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 104), dtype=float32)
  • training=True
  • mask=None

In [16]:
# model 2
# Build the model
model = keras.models.Sequential([
    keras.layers.Masking(mask_value=0.0, input_shape=(None, 1)),  # for variable-length sequences
    keras.layers.SimpleRNN(32),  # SimpleRNN with 32 units
    keras.layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_dataset, epochs=5,
                    validation_data=(val_dataset))

Epoch 1/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 266s 10ms/step - accuracy: 0.7691 - loss: 0.4878 - val_accuracy: 0.7860 - val_loss: 0.4799
Epoch 2/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 265s 10ms/step - accuracy: 0.7742 - loss: 0.4825 - val_accuracy: 0.7846 - val_loss: 0.4761
Epoch 3/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 266s 10ms/step - accuracy: 0.7754 - loss: 0.4819 - val_accuracy: 0.7830 - val_loss: 0.4791
Epoch 4/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 269s 10ms/step - accuracy: 0.7742 - loss: 0.4816 - val_accuracy: 0.7570 - val_loss: 0.4921
Epoch 5/5
26578/26578 ━━━━━━━━━━━━━━━━━━━━ 267s 10ms/step - accuracy: 0.7722 - loss: 0.4840 - val_accuracy: 0.7853 - val_loss: 0.4760


In [17]:
loss, accuracy = model.evaluate(test_dataset)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


5696/5696 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.7869 - loss: 0.4739
Loss:  0.4750058948993683
Accuracy:  0.7862943410873413


In [ ]:
#imbalanced data?
#data augmentation?

## Model trial #2, same data, new processing

In [18]:
import pandas as pd
from os import listdir
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Masking, InputLayer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.base import BaseEstimator


import pandas as pd
import glob

# Get all CSV file paths
csv_files = glob.glob("/sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project/final_data2/*.csv")

csv_files_sorted = sorted(csv_files)

# Read and concatenate all CSV files into a single DataFrame
dataframes = [pd.read_csv(file) for file in csv_files_sorted]
combined_df = pd.concat(dataframes, ignore_index=True)

# Handle categorical variables with Label Encoding
label_encoder = LabelEncoder()

# Encode categorical features
categorical_columns = ['insurance', 'race', 'marital_status', 'gender', 'value', 'amountuom', 'amount']
for col in categorical_columns:
    combined_df[col] = label_encoder.fit_transform(combined_df[col].astype(str))

# Encode target variable `Died` (binary classification: 0 = Alive, 1 = Died)
combined_df['Died'] = label_encoder.fit_transform(combined_df['Died'])

# Sort by Unique Stay and sequence number to get events in correct order
combined_df.sort_values(by=['Unique Stay', 'sequence_num'], inplace=True)

# Create sequences of features for each Unique Stay
sequence_data = []
sequence_labels = []

unique_stays = combined_df['Unique Stay'].unique()

# Iterate over unique stays
for stay in unique_stays:
    stay_data = combined_df[combined_df['Unique Stay'] == stay]
    stay_features = stay_data[['insurance', 'race', 'marital_status', 'gender', 'anchor_age', 'value', 'amount', 'amountuom']].values
    stay_label = stay_data['Died'].values[-1]  # Use the last event to define the label
    
    # Add the sequence and its corresponding label
    sequence_data.append(stay_features)
    sequence_labels.append(stay_label)

# Pad sequences to ensure uniform input length
sequence_data = pad_sequences(sequence_data, padding='post', dtype='float32', maxlen=50)  # Adjust maxlen as needed

# Convert labels to numpy array
sequence_labels = np.array(sequence_labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(sequence_data, sequence_labels, test_size=0.2, random_state=42)


In [28]:
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

Training set size: 4288
Testing set size: 1073


In [30]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])), 
    keras.layers.Masking(mask_value=0.0),  # for variable-length sequences
    keras.layers.SimpleRNN(32),  # SimpleRNN with 32 units
    keras.layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=5)    
    


Epoch 1/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9021 - loss: 0.4512
Epoch 2/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8999 - loss: 0.3256
Epoch 3/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9052 - loss: 0.3137
Epoch 4/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9011 - loss: 0.3226
Epoch 5/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9012 - loss: 0.3228


In [32]:
loss, accuracy = model.evaluate(X_test,y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9068 - loss: 0.3101
Loss:  0.31016674637794495
Accuracy:  0.9068033695220947


## Final Data used for the final (preliminary) model

In [4]:
import pandas as pd
from os import listdir
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator

import tensorflow as tf
#from tensorflow import keras

2024-11-10 17:45:32.348687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 17:45:32.367035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 17:45:32.372744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 17:45:32.386734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
data_path = "/sfs/gpfs/tardis/home/rmd9ev/Documents/MSDS/DS6050/Project/data/final_data"

# Files to read
data_files = listdir(data_path)

# Load data
data = pd.concat([pd.read_csv(data_path + '/' + file) for file in data_files[1:]])

In [6]:
data.head()

,Unique Stay,time_bucket,anchor_age,mortality,sequence_num,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Dobutamine,Dopamine,Epinephrine,Heart Rate,Inspired O2 Fraction,Lactic Acid,Norepinephrine,Platelet Count,Total Bilirubin
0,1000201323581541,2160-05-18 10:00:00,53.0,False,1.0,NaN,421.0,NaN,NaN,NaN,NaN,NaN,NaN,3.3,NaN,NaN,NaN
1,1000201323581541,2160-05-18 11:00:00,53.0,False,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN
2,1000201323581541,2160-05-18 12:00:00,53.0,False,4.0,NaN,384.0,NaN,NaN,NaN,NaN,NaN,NaN,3.1,NaN,NaN,NaN
3,1000201323581541,2160-05-18 13:00:00,53.0,False,7.0,NaN,311.0,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,252.0,NaN
4,1000201323581541,2160-05-18 14:00:00,53.0,False,11.0,94.0,332.0,1.1,NaN,NaN,NaN,80.0,100.0,NaN,NaN,254.0,NaN


In [7]:
data.isna().sum()

Unique Stay                          0
time_bucket                          0
anchor_age                           0
mortality                            0
sequence_num                         0
Arterial Blood Pressure mean    151250
Arterial O2 pressure            275099
Creatinine (serum)              288769
Dobutamine                      306916
Dopamine                        307026
Epinephrine                     301929
Heart Rate                       52536
Inspired O2 Fraction            269565
Lactic Acid                     290887
Norepinephrine                  293959
Platelet Count                  287295
Total Bilirubin                 305054
dtype: int64

In [8]:
na_data = ['Arterial Blood Pressure mean', 'Arterial O2 pressure', 'Creatinine (serum)','Dobutamine','Dopamine','Epinephrine','Heart Rate','Inspired O2 Fraction','Lactic Acid',
          'Norepinephrine','Platelet Count','Total Bilirubin']
for i in na_data:
    data[i].fillna(method = 'bfill', inplace= True)

In [9]:
data.isna().sum()

Unique Stay                         0
time_bucket                         0
anchor_age                          0
mortality                           0
sequence_num                        0
Arterial Blood Pressure mean        1
Arterial O2 pressure                9
Creatinine (serum)                  2
Dobutamine                      11625
Dopamine                         3858
Epinephrine                        35
Heart Rate                          0
Inspired O2 Fraction                0
Lactic Acid                         1
Norepinephrine                    100
Platelet Count                     15
Total Bilirubin                   113
dtype: int64

In [10]:
na_data = ['Arterial Blood Pressure mean','Arterial O2 pressure', 'Creatinine (serum)','Dobutamine','Dopamine','Epinephrine','Lactic Acid',
          'Norepinephrine','Platelet Count','Total Bilirubin']
for i in na_data:
    data[i].fillna(method = 'ffill', inplace= True)

In [11]:
data.isna().sum()

Unique Stay                     0
time_bucket                     0
anchor_age                      0
mortality                       0
sequence_num                    0
Arterial Blood Pressure mean    0
Arterial O2 pressure            0
Creatinine (serum)              0
Dobutamine                      0
Dopamine                        0
Epinephrine                     0
Heart Rate                      0
Inspired O2 Fraction            0
Lactic Acid                     0
Norepinephrine                  0
Platelet Count                  0
Total Bilirubin                 0
dtype: int64

In [12]:
data

,Unique Stay,time_bucket,anchor_age,mortality,sequence_num,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Dobutamine,Dopamine,Epinephrine,Heart Rate,Inspired O2 Fraction,Lactic Acid,Norepinephrine,Platelet Count,Total Bilirubin
0,1000201323581541,2160-05-18 10:00:00,53.0,False,1.0,94.0,421.0,1.1,13.272410,50.000001,0.448468,80.0,100.0,3.3,0.057591,252.0,0.8
1,1000201323581541,2160-05-18 11:00:00,53.0,False,3.0,94.0,384.0,1.1,13.272410,50.000001,0.448468,80.0,100.0,2.8,0.057591,252.0,0.8
2,1000201323581541,2160-05-18 12:00:00,53.0,False,4.0,94.0,384.0,1.1,13.272410,50.000001,0.448468,80.0,100.0,3.1,0.057591,252.0,0.8
3,1000201323581541,2160-05-18 13:00:00,53.0,False,7.0,94.0,311.0,1.1,13.272410,50.000001,0.448468,80.0,100.0,2.6,0.057591,252.0,0.8
4,1000201323581541,2160-05-18 14:00:00,53.0,False,11.0,94.0,332.0,1.1,13.272410,50.000001,0.448468,80.0,100.0,2.8,0.057591,254.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1655011221178124,2171-12-18 00:00:00,68.0,False,83.0,88.0,135.0,1.1,7.222779,50.893585,0.019991,86.0,35.0,0.9,0.079840,91.0,0.3
99996,1655011221178124,2171-12-18 01:00:00,68.0,False,87.0,90.0,135.0,1.1,7.222779,50.893585,0.019991,88.0,35.0,0.9,0.079840,91.0,0.3
99997,1655011221178124,2171-12-18 02:00:00,68.0,False,89.0,91.0,135.0,1.1,7.222779,50.893585,0.019991,87.0,35.0,0.9,0.079840,91.0,0.3
99998,1655011221178124,2171-12-18 03:00:00,68.0,False,93.0,88.0,135.0,1.1,7.222779,50.893585,0.019991,87.0,35.0,0.9,0.079840,91.0,0.3


In [13]:
label_encoder = LabelEncoder()
# Encode target variable `Died` (binary classification: 0 = Alive, 1 = Died)
data['mortality'] = label_encoder.fit_transform(data['mortality'])

# Sort by Unique Stay and sequence number to get events in correct order
data.sort_values(by=['Unique Stay', 'sequence_num'], inplace=True)

In [14]:
# Create sequences of features for each Unique Stay
sequence_data = []
sequence_labels = []

unique_stays = data['Unique Stay'].unique()

In [15]:
data = data.drop(columns=['time_bucket'])

In [16]:
for stay in unique_stays:
    stay_data = data[data['Unique Stay'] == stay]
    stay_features = stay_data[['anchor_age', 'mortality','Arterial Blood Pressure mean','Arterial O2 pressure','Creatinine (serum)',
                             'Dobutamine','Dopamine','Epinephrine','Heart Rate','Inspired O2 Fraction','Lactic Acid','Norepinephrine','Platelet Count',
                              'Total Bilirubin']].values
    stay_label = stay_data['mortality'].values[-1]  # Use the last event to define the label
    
    # Add the sequence and its corresponding label
    sequence_data.append(stay_features)
    sequence_labels.append(stay_label)

In [17]:
#sequence_data

In [18]:
import keras

In [19]:
#from tensorflow.keras.utils import pad_sequences
sequence_data = keras.utils.pad_sequences(sequence_data, padding='post', dtype='float32', maxlen=50)  # Adjust maxlen as needed

In [20]:
# Convert labels to numpy array
sequence_labels = np.array(sequence_labels)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(sequence_data, sequence_labels, test_size=0.2, random_state=42)

In [22]:
from keras import layers, Sequential

In [32]:
#run model with chosen hyperparameters
model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])), 
            keras.layers.Masking(mask_value=0.0),  # for variable-length sequences
            keras.layers.SimpleRNN(32,dropout=0.2),  # SimpleRNN with 32 units
            keras.layers.Dense(1, activation='sigmoid') 
])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy','AUC'])
history = model.fit(X_train,y_train, epochs=10)

Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - AUC: 0.4862 - accuracy: 0.3499 - loss: 1.0026
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5881 - accuracy: 0.9862 - loss: 0.1581
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5616 - accuracy: 0.9869 - loss: 0.0922
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.6515 - accuracy: 0.9890 - loss: 0.0668
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.6593 - accuracy: 0.9883 - loss: 0.0651
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5723 - accuracy: 0.9862 - loss: 0.0751
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.6680 - accuracy: 0.9877 - loss: 0.0651
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5791 - accuracy: 0.9896 - loss: 0.0580
Epoch 9/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5309 - accuracy: 0.9907 - loss: 0.0548
Epoch 10/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5980 - accuracy: 0.9885 - loss: 0.063

In [34]:
loss,accuracy,auc = model.evaluate(X_test,y_test)
print("loss", loss)
print("accuracy", accuracy)
print("auc", auc)

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5527 - accuracy: 0.9865 - loss: 0.0691  
loss 0.08263782411813736
accuracy 0.98392653465271
auc 0.6034339070320129


In [35]:
#run model and search for best parameters
class KerasModelWrapper(BaseEstimator):
    def __init__(self, learning_rate=0.001, dropout=0.2, epochs=10, batch_size=32):
        self.learning_rate = learning_rate
        self.dropout = dropout
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
    def create_model(self):
        model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])), 
            keras.layers.Masking(mask_value=0.0),  # for variable-length sequences
            keras.layers.SimpleRNN(32,dropout=self.dropout),  # SimpleRNN with 32 units
            keras.layers.Dense(1, activation='sigmoid') 
])
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate), loss='binary_crossentropy', metrics=['accuracy','AUC'])
        return model
    def fit(self, X, y):
        self.model = self.create_model()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        return self
    def predict(self, X):
        return self.model.predict(X)
    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]

In [36]:
model = KerasModelWrapper()

In [37]:
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout': [0.2, 0.3, 0.5],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 50]
}

In [38]:
random_search = RandomizedSearchCV(estimator=model,param_distributions=param_grid, n_iter=10, cv=3, verbose=2, n_jobs=-1, random_state=42)

In [39]:
random_search_result = random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


2024-11-10 18:16:29.451061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:16:29.454298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:16:29.469305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:16:29.472602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:16:29.474865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory

[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.001; total time=  27.4s
[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.001; total time=  22.6s
[CV] END batch_size=16, dropout=0.2, epochs=10, learning_rate=0.001; total time=  20.1s
[CV] END batch_size=16, dropout=0.5, epochs=20, learning_rate=0.01; total time=  38.7s
[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.01; total time=  25.0s
[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.001; total time=  28.0s
[CV] END batch_size=16, dropout=0.2, epochs=10, learning_rate=0.001; total time=  18.8s
[CV] END batch_size=16, dropout=0.2, epochs=10, learning_rate=0.001; total time=  20.9s
[CV] END batch_size=16, dropout=0.5, epochs=20, learning_rate=0.01; total time=  34.5s
[CV] END batch_size=16, dropout=0.5, epochs=20, learning_rate=0.01; total time=  33.6s


2024-11-10 18:18:59.174582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:18:59.194117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:18:59.200123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 18:19:00.189152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:19:00.207102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factor

[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.01; total time=  28.1s
[CV] END batch_size=16, dropout=0.5, epochs=10, learning_rate=0.001; total time=  20.8s
[CV] END batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01; total time=  15.2s
[CV] END batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01; total time=   9.1s
[CV] END batch_size=16, dropout=0.3, epochs=10, learning_rate=0.01; total time=  15.4s


2024-11-10 18:20:36.614233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:20:36.632989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:20:36.638758: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[CV] END batch_size=32, dropout=0.2, epochs=20, learning_rate=0.01; total time=  26.4s
[CV] END batch_size=16, dropout=0.5, epochs=10, learning_rate=0.001; total time=  19.7s
[CV] END batch_size=16, dropout=0.5, epochs=10, learning_rate=0.001; total time=  21.1s
[CV] END batch_size=32, dropout=0.2, epochs=10, learning_rate=0.01; total time=   9.7s
[CV] END batch_size=16, dropout=0.3, epochs=10, learning_rate=0.01; total time=  16.5s


2024-11-10 18:20:40.987011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:20:41.006893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:20:41.012722: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/rmd9ev/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1731280843.751277  275387 service.cc:146] XLA service 0x55beac4fe140 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:17312

[CV] END batch_size=16, dropout=0.3, epochs=10, learning_rate=0.01; total time=  18.4s
[CV] END batch_size=64, dropout=0.3, epochs=50, learning_rate=0.01; total time=  25.4s
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  36.1s
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  39.7s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  35.9s
[CV] END batch_size=64, dropout=0.3, epochs=50, learning_rate=0.01; total time=  26.7s
[CV] END batch_size=64, dropout=0.3, epochs=50, learning_rate=0.01; total time=  27.0s
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  37.3s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  37.7s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  29.9s


In [28]:
print("Best Hyperparameters:", random_search_result.best_params_)
print("Best Accuracy:", random_search_result.best_score_)

Best Hyperparameters: {'learning_rate': 0.001, 'epochs': 20, 'dropout': 0.2, 'batch_size': 32}
Best Accuracy: nan


In [29]:
best_model = random_search_result.best_estimator_
test_acc = best_model.score(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.9839


In [27]:
best_model = random_search_result.best_estimator_

In [28]:
model_save_path = 'RNN_best_model.keras'
best_model.model.save(model_save_path)

In [29]:
print(f"Best model saved at: {model_save_path}")

Best model saved at: RNN_best_model.keras


In [30]:
test_acc = best_model.score(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.9839


In [35]:
loaded_model = tf.keras.models.load_model('RNN_best_model.keras')
loaded_test_loss, loaded_test_acc = loaded_model.evaluate(X_test, y_test, verbose=0)

In [36]:
print(f"Loaded Model Test Accuracy: {loaded_test_acc:.4f}")

Loaded Model Test Accuracy: 0.9839
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  39.6s
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  41.8s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  44.5s
[CV] END batch_size=64, dropout=0.3, epochs=50, learning_rate=0.01; total time=  25.8s
[CV] END batch_size=16, dropout=0.2, epochs=20, learning_rate=0.01; total time=  39.5s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  38.0s
[CV] END batch_size=16, dropout=0.3, epochs=20, learning_rate=0.001; total time=  33.2s
